# Python Enhancement Proposal: Stuyle Guide for Python Code

**Exercise 1 – Python Syntax & Naming Conventions**

In [ ]:
# before
import os,sys

def Load_Data(FilePath:str)->list:
    "load jsonl file and return list of lines"
    if FilePath == None:
        raise ValueError("filepath can not be None")

    data=[]
    with open(FilePath, "r") as f:
        for line in f.readlines():
            if len(line.strip()) == 0: continue
            data.append( line )

    print("loaded %s lines from file" % len(data))
    return data


def filterlines(lines, Min_Length =10):
    "filter by min length"
    result = []
    for l in lines:
        if len(l) > Min_Length:
            result.append(l)
    return result


In [ ]:
# after
import os
import sys

def load_data(file_path:str)->list[str]:
    "load jsonl file and return list of lines"
    if file_path is None:
        raise ValueError(f"{file_path} can not be None")

    data=[]
    with open(file_path, "r") as f:
        for line in f:
            if len(line.strip()) == 0: continue
            data.append(line)

    print(f"loaded {len(data)} lines from file")
    return data


def filter_lines(lines: list[str], min_Length =10)->list[str]:
    "filter by min length"
    return [ l for l in lines if len(l) > min_Length ]


**Exercise 2 – Typos & Buggy Logic**

In [ ]:
# before
def normalize_scores(scors: list[float]) -> list[float]:
    "Normalize a list of scores to 0-1 range"
    min_score = min(scors)
    max_score = max(score for score in scors)
    span = max_score - min_score

    if span == 0:
        return [0.0 for s in scores]

    normd = []
    for score in scores:
        norm = (scor - min_score) / span
        normd.append(norm)

    retrun normd


In [ ]:
# after
from typing import List

def normalize_scores(scores: List[float]) -> List[float]:
    "Normalize a list of scores to 0-1 range"
    if not scores:
        raise ValueError("scores can not be None")
    min_score = min(scores)
    max_score = max(scores)
    span = max_score - min_score

    if span == 0:
        return [0.0 for _ in scores]

    return [(s - min_score)/span for s in scores]


**Exercise 3 – Async Syntax & Blocking Code**

In [ ]:
# before
import asyncio
import time
from typing import List

async def fetch_user_profile(user_id: str) -> dict:
    """Fake network call"""
    time.sleep(0.5)  # simulate IO
    return {"userId": user_id, "status": "ok"}

async def fetch_all_users(ids: List[str]) -> list[dict]:
    profiles = []
    for id in ids:
        profile = fetch_user_profile(id)
        profiles.append(profile)
    return await asyncio.gather(*profiles)

async def main():
    users = ["u1", "u2", "u3"]
    results = await fetch_all_users(users)
    print("done:", results)

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    loop.close()


In [ ]:
# after
import asyncio
import time
from typing import List, Dict

async def fetch_user_profile(user_id: str) -> Dict[str, str]:
    """Fake network call"""
    # time.sleep(0.5)  # simulate IO
    await asyncio.sleep(0.5)
    return {"userId": user_id, "status": "ok"}

async def fetch_all_users(ids: List[str]) -> List[Dict[str, str]]:
    profiles = [fetch_user_profile(id) for id in ids]
    return await asyncio.gather(*profiles)

async def main():
    users = ["u1", "u2", "u3"]
    results = await fetch_all_users(users)
    print("done:", results)

if __name__ == "__main__":
    asyncio.run(main())


**Exercise 4 – Mistral API (Sync) Review**

In [ ]:
# before
import os
from mistralai import Mistral

def call_mistral(prompt: str) -> str:
    client = Mistral(api_key=os.environ["MISTRAL_KEY"])
    model_name = "mistral-medium-latest"

    response = client.chat.completions.create(
        model=model_name,
        message=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return response.choices[0].message["content"]


In [ ]:
# after
import os
from typing import List
from mistralai import Mistral

client = Mistral(api_key=os.environ["MISTRAL_KEY"])

DEFAULT_MODEL = "mistral-medium-latest"

def call_mistral(prompt: str, model_name: str = DEFAULT_MODEL) -> str:

    response = client.chat.complete(
        model=model_name,
        message=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return response.choices[0].message.content

**Exercise 5 – Mistral API (Async) + Async Pitfalls**

In [ ]:
# before
import os
import asyncio
from mistralai import Mistral

async def chat_once(user_msg: str) -> str:
    client = Mistral(api_key=os.getenv("MISTRAL_API_KEY", ""))

    async with client:
        res = client.chat.complete_async(
            model="mistral-small-latest",
            messages={
                "role": "user",
                "content": user_msg
            },
            stream=False
        )
        return res.choices[0].message.content

def main():
    answer = asyncio.run(chat_once("hello!"))
    print(answer)

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


In [ ]:
# after
import os
import asyncio
from mistralai import Mistral

DEFAULT_MODEL = "mistral-small-latest"

async def chat_once(user_msg: str, model: str = DEFAULT_MODEL) -> str:

    async with Mistral(api_key=os.getenv("MISTRAL_API_KEY", "")) as client:
        res =  await client.chat.complete_async(
            model=DEFAULT_MODEL,
            messages=[
                {
                "role": "user",
                "content": user_msg
                }
            ],
            stream=False
        )
        return res.choices[0].message.content

async def main():
    answer = await chat_once("hello!")
    print(answer)

if __name__ == "__main__":
    asyncio.run(main())

**Exercise 1 – Messy Data Pipeline + Confusing Types**

In [ ]:
# before
# data_pipeline.py

import json,os,asyncio,time
from typing import Dict, List, Any

async def load_json_file(path:str)->list:
    f = open(path)
    txt = f.read()
    lines = txt.split("\n")
    data = []
    for l in lines:
        if l.strip()=="":
            continue
        try:
            data.append(json.loads(l))
        except:
            print("bad json:", l)
    f.close()
    return data

def filter_users(objs, minage=18, maxage=99, country=None):
    res=[]
    for x in objs:
        # age can be str
        age = int(x.get("age", 0))
        if age<minage or age>maxage:
            continue
        if country and x["country"]!=country:
            continue
        res.append(x)
    return res

async def enrich_with_delay(users:List[Dict[str,Any]]):
    enriched=[]
    for u in users:
        time.sleep(0.05)  # simulate external service
        u["score"]=u.get("score",0)+1
        enriched.append(u)
    return enriched

async def run(path, country):
    # can be called inside FastAPI endpoint
    data = await load_json_file(path)
    filtered = filter_users(data, country=country)
    result = await enrich_with_delay(filtered)
    return result


In [ ]:
# after
# data_pipeline.py
import asyncio
import json
import os
import time
from typing import Dict, List, Any, Optional, Iterable

def load_json_file(path:Optional[str])->List[Dict[str,Any]]:
    if not path:
        raise ValueError('Please provide a file path!')
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    with open(path, 'r', encoding="utf-8") as f:
        records = []
        for line_no, line in enumerate(f, start=1):
            line = line.strip()
            if line=="":
                continue
            try:
                record = json.loads(line)
            except json.decoder.JSONDecodeError:
                print(f'bad json in line {line_no}: {line}')
                continue
            records.append(record)
    return records

def filter_users(records: Iterable[Dict[str, Any]], minage:int=18, maxage:int =99, country:Optional[str] =None)->List[Dict[str,Any]]:
    res=[]
    for record in records:
        # age can be str
        age = record.get("age", 0)
        try:
            age = int(age)
        except:
            age = 0
        if age<minage or age>maxage:
            continue
        if country and record.get("country") !=country:
            continue
        res.append(record)
    return res

async def enrich_with_delay(users:List[Dict[str,Any]])->List[Dict[str,Any]]:
    enriched=[]
    for u in users:
        await asyncio.sleep(0.05)  # simulate external service
        score = u.get("score",0)
        try:
            u["score"] = int(score) + 1
        except:
            u["score"] = 1
        enriched.append(u)
    return enriched

async def run(path:str, country:str=None)->List[Dict[str,Any]]:
    # can be called inside FastAPI endpoint
    data = load_json_file(path)
    filtered = filter_users(data, country=country)
    result = await enrich_with_delay(filtered)
    return result


In [ ]:
# async file reading
import json
import os
from typing import List, Dict, Any
import aiofiles


async def load_json_file(path: str) -> List[Dict[str, Any]]:
    if not path:
        raise ValueError("Please provide a non-empty file path!")

    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")

    records: List[Dict[str, Any]] = []

    async with aiofiles.open(path, "r", encoding="utf-8") as f:
        line_no = 0
        async for line in f:
            line_no += 1
            line = line.strip()
            if not line:
                continue
            try:
                records.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"Bad JSON on line {line_no}: {line!r}")

    return records


**Exercise 2 – “Helper” Mistral Wrapper That’s Dangerous in Production**

In [ ]:
# before
# mistral_helper_before.py
import os, traceback
from mistralai import Mistral

def ask_mistral(prompt: str, system_prompt: str = None):
    client = Mistral(api_key=os.getenv("MISTRAL_KEY"))
    try:
        if system_prompt:
            msgs = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        else:
            msgs = [{"role": "user", "content": prompt}]

        print("sending to model:", msgs)

        resp = client.chat.complete(
            model="mistral-tiny-latest",
            messages=msgs,
            temperature=0.9,
            max_tokens=4096,
            safe_mode=False
        )

        return resp.choices[0].message.content
    except Exception as e:
        print("ERROR:", e)
        traceback.print_exc()
        return str(e)


In [ ]:
# after
"""
mistral_helper_before.py

Small educational wrapper around the Mistral Python client.

- Shows how to:
    * structure a client class
    * wrap low-level exceptions in your own error type
    * build messages
    * add basic logging and configuration
"""

import os
from typing import List, Dict, Optional

from mistralai import Mistral


# ----------------------------
# Configuration constants
# ----------------------------

# Default model you want to call. You can change this in one place.
DEFAULT_MODEL = "mistral-tiny-latest"

# Environment variable name for the API key.
# You can configure this in your shell / .env file.
API_KEY_ENV = "MISTRAL_API_KEY"


# ----------------------------
# Custom error type
# ----------------------------

class MistralClientError(Exception):
    """
    High-level wrapper for errors when calling the Mistral API.

    Why define this?
    - You usually don't want to leak raw SDK or HTTP exceptions all over your code.
    - A custom error type lets you:
        * add context (which model? what kind of request?)
        * catch it specifically in higher-level code: `except MistralClientError: ...`

    Attributes
    ----------
    message : str
        Human-readable description of the error.
    original_exc : BaseException | None
        The underlying exception raised by the SDK / HTTP client, if any.
    model : str | None
        Model name used for the request (useful to debug config issues).
    request_payload : dict | None
        Optional metadata snapshot (e.g. model + number of messages).
        NOTE: In real production code, be careful not to include PII or secrets here.
    """

    def __init__(
        self,
        message: str,
        *,
        original_exc: Optional[BaseException] = None,
        model: Optional[str] = None,
        request_payload: Optional[dict] = None,
    ) -> None:
        # Call base Exception __init__ so str(e) works.
        super().__init__(message)

        self.message = message
        self.original_exc = original_exc
        self.model = model
        self.request_payload = request_payload

    def __str__(self) -> str:
        """
        String representation used when you print the exception.
        We add some extra context if available.
        """
        base = self.message

        if self.model:
            base += f" [model={self.model}]"

        if self.original_exc is not None:
            base += f" (caused by {type(self.original_exc).__name__}: {self.original_exc})"

        return base


# ----------------------------
# Mistral client wrapper
# ----------------------------

class MistralClient:
    """
    Simple wrapper around `mistralai.Mistral` for chat completion.

    Responsibilities:
    - Own the underlying `Mistral` SDK client.
    - Build messages from prompt + optional system prompt.
    - Handle common options (temperature, max_tokens, safe_mode).
    - Wrap errors into `MistralClientError`.

    This is a *sync* wrapper (using `client.chat.complete`).
    You could add an async version later if needed.
    """

    def __init__(
        self,
        api_key: Optional[str] = None,
        model: str = DEFAULT_MODEL,
        log_prompts: bool = False,
    ) -> None:
        """
        Parameters
        ----------
        api_key : str | None
            The Mistral API key. If None, we'll read it from `API_KEY_ENV`.
        model : str
            Default model name to use for requests.
        log_prompts : bool
            Whether to log prompt metadata (NOT the full prompt) to stdout.
        """
        # Prefer explicit `api_key`, fall back to environment variable.
        self.api_key = api_key or os.getenv(API_KEY_ENV)

        if not self.api_key:
            raise ValueError(
                f"Mistral API key not found. Please pass `api_key` or set env {API_KEY_ENV}"
            )

        self.model = model
        self.log_prompts = log_prompts

        # Create underlying SDK client once and reuse it.
        self._client = Mistral(api_key=self.api_key)

    # ------------- internal helper -------------

    def _build_messages(
        self,
        prompt: str,
        system_prompt: Optional[str],
    ) -> List[Dict[str, str]]:
        """
        Convert a user prompt + optional system prompt into
        the `messages=[...]` format expected by the Mistral chat API.

        Example output:
        [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello!"}
        ]
        """
        messages: List[Dict[str, str]] = []

        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})

        messages.append({"role": "user", "content": prompt})

        return messages

    # ------------- public method -------------

    def ask(
        self,
        prompt: str,
        system_prompt: Optional[str] = None,
        *,
        temperature: float = 0.7,
        max_tokens: int = 1024,
        safe_mode: bool = True,
    ) -> str:
        """
        Send a prompt (and optional system prompt) to Mistral and return the text response.

        This is a convenience method for simple single-turn calls.

        Parameters
        ----------
        prompt : str
            The user prompt.
        system_prompt : str | None
            Optional system message that sets behavior or instructions.
        temperature : float
            Sampling temperature; higher is more random.
        max_tokens : int
            Maximum number of tokens to generate.
        safe_mode : bool
            Whether to enable safety filters (depends on Mistral's API behavior).

        Returns
        -------
        str
            The model's response text.

        Raises
        ------
        MistralClientError
            If there is any problem calling the Mistral API.
        """
        messages = self._build_messages(prompt, system_prompt)

        if self.log_prompts:
            # In real code, use logging instead of print, and avoid logging full text for PII.
            print(
                "[MistralClient] Sending chat completion request:",
                f"model={self.model}, num_messages={len(messages)}",
            )

        try:
            # Call the underlying SDK.
            # NOTE: This is sync; for async you would use `await client.chat.complete_async(...)`.
            resp = self._client.chat.complete(
                model=self.model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                safe_mode=safe_mode,
            )
        except Exception as exc:
            # Build a small payload with non-sensitive context for debugging.
            payload_meta = {
                "model": self.model,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "safe_mode": safe_mode,
                "num_messages": len(messages),
            }

            # Wrap low-level exception in our custom error type.
            # The `from exc` keeps traceback chaining (`__cause__`).
            raise MistralClientError(
                "Error calling Mistral chat completion",
                original_exc=exc,
                model=self.model,
                request_payload=payload_meta,
            ) from exc

        # Extract the text content from the first choice.
        return resp.choices[0].message.content


# ----------------------------
# Example usage (manual test)
# ----------------------------

if __name__ == "__main__":
    """
    If you run this file directly:
    - make sure you have `MISTRAL_API_KEY` set in your environment
    - it will send a small prompt and print the reply
    """
    client = MistralClient(log_prompts=True)

    try:
        answer = client.ask("Say hello in one short sentence.")
        print("Model answer:", answer)
    except MistralClientError as e:
        # Here you see how your custom error behaves.
        print("MistralClientError occurred:", e)
        if e.request_payload:
            print("Debug payload:", e.request_payload)


**Exercise 3 – Async + CPU Work + Race Conditions**

In [ ]:
# before
# scoring_service.py

import asyncio, math
from typing import List, Dict

async def slow_score(user: Dict) -> float:
    """Fake scoring logic mixing IO and CPU."""
    # simulate external feature fetch
    await asyncio.sleep(0.1)
    base = user.get("base", 0)
    try:
        base = int(base)
    except:
        raise ValueError(f'{base} cannot be converted to int')
    # heavy-ish CPU work
    s = 0
    for i in range(1, 100000):
        s += math.log(i + base + 1)
    return s

async def compute_scores(users: List[Dict]) -> None:
    tasks = {}
    for u in users:
        uid = u.get("id")
        if not uid or uid in GLOBAL_CACHE:
            continue
        # BUG: no isolation
        GLOBAL_CACHE[uid] = 0.0
        t = await slow_score(u)
        tasks[uid] = t
    return tasks

    # gather but ignore order
    for uid, t in tasks:
        GLOBAL_CACHE[uid] = await t

async def main():
    users = [{"id":"u1","base":3},{"id":"u2","base":5},{"id":"u1","base":7}]
    res = await compute_scores(users)
    print(res)


In [ ]:
# after
# scoring_service.py

import asyncio, math
from typing import List, Dict

def base_score_process(base: int)->float:
    s = 0
    for i in range(1, 100000):
        s += math.log(i + base + 1)
    return s

async def slow_score(user: Dict[str, Any]) -> float:
    """Fake scoring logic mixing IO and CPU."""
    # simulate external feature fetch
    await asyncio.sleep(0.1)
    base = user.get("base", 0)
    # heavy-ish CPU work
    res = await asyncio.to_thread(base_score_process, base)
    return res

async def compute_scores(users: List[Dict]) -> Dict[str, float]:
    res = {}
    uid_list = []
    score_list = []
    for u in users:
        uid = u["id"]
        uid_list.append(uid)
        score_list.append(asyncio.create_task(slow_score(u)))
    score_res = await asyncio.gather(*score_list, return_exceptions=True)
    for uid, score in zip(uid_list, score_res):
        if isinstance(score, Exception):
            print(f'cannot compute score for {uid}: {score}')
            res[uid] = 0.0
        else:
            res[uid] = score
    return res

async def main():
    users = [{"id":"u1","base":3},{"id":"u2","base":5},{"id":"u3","base":7}]
    res = await compute_scores(users)
    print(res)

if __name__ == "__main__":
    asyncio.run(main())



**Exercise 4 – Dataclass Misuse, Defaults, and Type Weirdness**

In [ ]:
# before
# models.py

from dataclasses import dataclass, field
from typing import List, Dict, Any

@dataclass
class ChatSession:
    userId: str
    history: List[Dict[str, str]] = []
    meta: Dict[str, Any] = field(default_factory=dict)
    system_prompt: str = None
    temperature: float = 0.8

    def add_utt(self, role, content):
        self.history.append({"role": role, "content": content})

    def trim(self, n):
        """Keep last n messages."""
        if n<0:
            return
        self.history = self.history[-n:]

    def to_mistral_messages(self):
        msgs = []
        if self.system_prompt:
            msgs.append({"role": "system", "content": self.system_prompt})
        msgs += self.history
        return msgs


In [ ]:
# after
# models.py

from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Literal

Role = Literal["system", "user", "assistant"]

@dataclass
class ChatSession:
    user_id: str
    history: List[Dict[str, str]] = field(default_factory=list)
    meta: Dict[str, Any] = field(default_factory=dict)
    system_prompt: Optional[str] = None
    temperature: float = 0.8

    def add_utterance(self, role:Role, content:str)->None:
        if role not in ('user', 'assistant', 'user'):
            raise ValueError('role can only be \"system\", \"assistant\" or \"user\"!')
        self.history.append({"role": role, "content": content})

    def trim_history(self, n:int)->None:
        """Keep last n messages."""
        if n<0:
            return
        self.history = self.history[-n:]

    def to_mistral_messages(self)->List[Dict[str, Any]]:
        msgs: List[Dict[str, Any]] = []
        if self.system_prompt:
            msgs.append({"role": "system", "content": self.system_prompt})
        msgs += self.history
        return msgs


**Exercise 1 – Async LLM Client Class (Messy)**

In [ ]:
# llm_client.py (junior version)

import os, asyncio, time
from typing import Any
from mistralai import Mistral

class LLMClient:
    def __init__(self, api_key: str | None = None, model="mistral-tiny-latest"):
        if api_key is None:
            api_key = os.getenv("MISTRAL_API_KEY")
        self.api_key = api_key
        self.model = model
        # create client immediately
        self.client = Mistral(api_key=self.api_key)
        self.total_calls = 0
        self.last_response: Any = None

    async def _sleep(self, seconds: float):
        # simulate network jitter (BAD: blocking)
        time.sleep(seconds)

    async def make_messages(self, prompt: str, system: str | None = None):
        msgs = []
        if system is not None:
            msgs.append({"role": "system", "content": system})
        msgs.append({"role": "user", "content": prompt})
        return msgs

    async def chat(self, prompt: str, system: str | None = None) -> str:
        # randomly sleep a bit
        await self._sleep(0.1)

        msgs = await self.make_messages(prompt, system)

        try:
            # forgot to await
            resp = self.client.chat.complete_async(
                model=self.model,
                messages=msgs,
            )
        except Exception as e:
            print("LLM error:", e)
            return "ERROR"

        self.total_calls += 1
        self.last_response = resp
        # assume dict-style
        return resp.choices[0].message["content"]


async def main():
    c = LLMClient()
    ans = await c.chat("hello", "You are helpful.")
    print(ans)

if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# llm_client.py (junior version)

import os, asyncio, time
from typing import Any, Optional, Dict, Optional
from mistralai import Mistral

DEFAULT_MODEL = "mistral-tiny-latest"

class LLMClient:
    def __init__(self, api_key: Optional[str] = None, model:str=DEFAULT_MODEL)->None:
        if api_key is None:
            api_key = os.getenv("MISTRAL_API_KEY")
        self.api_key = api_key
        if not self.api_key:
            raise ValueError("MISTRAL_API_KEY must be set!")
        self.model = model
        # create client immediately
        self.client = Mistral(api_key=self.api_key)
        self.total_calls:int = 0

    async def sleep(self, seconds: float)->None:
        # simulate network jitter (BAD: blocking)
        await asyncio.sleep(seconds)

    def make_messages(self, prompt: str, system: Optional[str] = None)->List[Dict[str, Any]]:
        msgs: List[Dict[str, Any]] = []
        if system is not None:
            msgs.append({"role": "system", "content": system})
        msgs.append({"role": "user", "content": prompt})
        return msgs

    async def chat(self, prompt: str, system: Optional[str] = None) -> str:
        # randomly sleep a bit
        await self.sleep(0.1)

        msgs = self.make_messages(prompt, system)

        try:
            # forgot to await
            resp = await self.client.chat.complete_async(
                model=self.model,
                messages=msgs,
            )
        except Exception as e:
            print("LLM error:", e)
            return "ERROR"

        self.total_calls += 1
        # assume dict-style
        return resp.choices[0].message.content


async def main():
    c = LLMClient()
    ans = await c.chat("hello", "You are helpful.")
    print(ans)

if __name__ == "__main__":
    asyncio.run(main())


**Exercise 2 – FastAPI + Async Client + Session Class**

In [ ]:
# session_service.py (junior version)

import asyncio
from typing import Dict, Any
from fastapi import FastAPI
from pydantic import BaseModel
from mistralai import Mistral

app = FastAPI()

# global stuff
SESSIONS: Dict[str, "Session"] = {}
client = Mistral(api_key="")  # FIXME: empty key


class ChatRequest(BaseModel):
    session_id: str
    user_message: str


class ChatResponse(BaseModel):
    session_id: str
    reply: str
    history: list[dict]


class Session:
    def __init__(self, session_id: str):
        self.id = session_id
        self.history: list[dict] = []
        self.model = "mistral-tiny-latest"

    async def add_user_message(self, content: str):
        self.history.append({"role": "user", "content": content})

    async def add_assistant_message(self, content: str):
        self.history.append({"role": "assistant", "content": content})

    async def to_messages(self):
        # no system prompt support, but could add later
        return self.history

    async def call_model(self) -> str:
        msgs = await self.to_messages()
        # missing await
        resp = client.chat.complete_async(
            model=self.model,
            messages=msgs,
        )
        # assume dict style
        return resp.choices[0].message["content"]


async def get_or_create_session(session_id: str) -> Session:
    if session_id in SESSIONS:
        return SESSIONS[session_id]
    s = Session(session_id)
    SESSIONS[session_id] = s
    return s


@app.post("/chat", response_model=ChatResponse)
async def chat(req: ChatRequest):
    # BAD: manually manage event loop inside FastAPI
    loop = asyncio.get_event_loop()

    session = await get_or_create_session(req.session_id)
    await session.add_user_message(req.user_message)

    # weird: use run_until_complete in async endpoint
    reply = loop.run_until_complete(session.call_model())
    await session.add_assistant_message(reply)

    return ChatResponse(
        session_id=session.id,
        reply=reply,
        history=session.history,
    )


In [ ]:
# session_service.py (junior version)

import asyncio
from typing import Dict, Any
from fastapi import FastAPI
from pydantic import BaseModel
from mistralai import Mistral

app = FastAPI()

# global stuff
SESSIONS: Dict[str, "Session"] = {}
client = Mistral(api_key="")  # FIXME: empty key


class ChatRequest(BaseModel):
    session_id: str
    user_message: str


class ChatResponse(BaseModel):
    session_id: str
    reply: str
    history: list[dict]

DEFAULT_MODEL = "mistral-tiny-latest"

class Session:
    def __init__(self, session_id: str, model:str=DEFAULT_MODEL)->None:
        self.id = session_id
        self.history: List[Dict[str, str]] = []
        self.model = model

    def add_user_message(self, content: str)->None:
        self.history.append({"role": "user", "content": content})

    def add_assistant_message(self, content: str)->None:
        self.history.append({"role": "assistant", "content": content})

    def to_messages(self)->List[Dict[str, str]]:
        # no system prompt support, but could add later
        return self.history

    async def call_model(self) -> str:
        msgs: List[Dict[str, str]] = self.to_messages()
        # missing await
        resp = client.chat.complete_async(
            model=self.model,
            messages=msgs,
        )
        # assume dict style
        return resp.choices[0].message["content"]


async def get_or_create_session(session_id: str) -> Session:
    if session_id in SESSIONS:
        return SESSIONS[session_id]
    s = Session(session_id)
    SESSIONS[session_id] = s
    return s


@app.post("/chat", response_model=ChatResponse)
async def chat(req: ChatRequest):
    # BAD: manually manage event loop inside FastAPI
    loop = asyncio.get_event_loop()

    session = await get_or_create_session(req.session_id)
    await session.add_user_message(req.user_message)

    # weird: use run_until_complete in async endpoint
    reply = loop.run_until_complete(session.call_model())
    await session.add_assistant_message(reply)

    return ChatResponse(
        session_id=session.id,
        reply=reply,
        history=session.history,
    )
